In [1]:
from scienceworld import ScienceWorldEnv
from sources.agent import BDIAgent
from sources.scienceworld import parse_observation

from sources.bdi_components.belief import State
from sources.bdi_components.inference import NLIModel
from sources.bdi_components.plans import PlanLibrary

In [2]:
#hg_model_hub_name = "alisawuffles/roberta-large-wanli"
hg_model_hub_name = "ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli"
nli_model = NLIModel(hg_model_hub_name)

Some weights of the model checkpoint at ynie/roberta-large-snli_mnli_fever_anli_R1_R2_R3-nli were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [9]:
# MAIN GOAL

plan_a = """
    IF your goal is to grow a orange plant from seed THEN
        move to living room,
        pick up seed jar,
        move to outside,
        collect soil,
        move to greenhouse,
        plant the orange seed,
        focus on orange seed in flower pot 1,
        water plants,
        focus on orange seed in flower pot 1
"""

plan_b = """
    IF your goal is to collect soil CONSIDERING you are seeing a shovel AND you are in outside THEN
        pick up shovel,
        use shovel in inventory on ground,
        pick up soil
"""

plan_c = """
IF your goal is to water plants CONSIDERING you are seeing a flower pot AND you see a sink THEN
    activate sink,
    move jug to sink,
    pour jug into flower pot 1,
    deactivate sink,
    wait1,
    wait1,
    wait1,
    wait1,
    wait1,
    activate sink,
    move jug to sink,
    pour jug into flower pot 1,
    deactivate sink,
    wait1,
    wait1,
    wait1,
    wait1,
    wait1,
    activate sink,
    move jug to sink,
    pour jug into flower pot 1,
    deactivate sink,
    wait1,
    wait1,
    wait1,
    wait1,
    wait1
"""

plan_d = """
    IF your goal is to plant the orange seed CONSIDERING you are seeing a flower pot AND you have a orange seed in your inventory AND you have soil in your inventory THEN
        move soil in inventory to flower pot 1,
        move orange seed in seed jar to flower pot 1,
        0
"""

all_plans = [plan_a, plan_b, plan_c, plan_d]

pl = PlanLibrary()
pl.load_plans_from_strings(all_plans)  # load plans from strings above
pl.load_plans_from_file("plans.txt")  # load plans from file
print(pl.plans.keys())

dict_keys(['grow a orange plant from seed', 'collect soil', 'water plants', 'plant the orange seed', 'move to art studio', 'move to bedroom', 'move to greenhouse', 'move to kitchen', 'move to living room', 'move to workshop', 'move to outside', 'move to foundry', 'move to bathroom', 'move to hallway'])


In [10]:
env = ScienceWorldEnv("", "", envStepLimit=100)

#root_event = 'use chemistry to create green paint'

task = 'grow-plant'
env.load(task, 0)
#randVariationIdx = env.getRandomVariationTest()
randVariationIdx = 98
env.load(task, randVariationIdx)

goal = env.getTaskDescription().split('.')[0].replace("Your task is to", "").strip()

print(f"Task Name: " + 'boil' + " variation " + str(randVariationIdx))
print("Task Description: " + str(env.getTaskDescription()))

# Reset the environment
observation, info = env.reset()
# initial state
observation, reward, isCompleted, info = env.step('look around')
current_state = parse_observation(observation=observation, inventory=info['inv'], look_around=info['look'],
                                  task=goal, valid_actions=info['valid'])

goal

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Task Name: boil variation 98
Task Description: Your task is to grow a orange plant from seed. Seeds can be found in the living room. First, focus on a seed. Then, make changes to the environment that grow the plant until it reaches the reproduction life stage.


'grow a orange plant from seed'

In [11]:
def step_function(action: str) -> State:
    observation, reward, isCompleted, info = env.step(action)
    updated_state = parse_observation(observation=observation,
                                      inventory=info['inv'],
                                      look_around=info['look'],
                                      task=goal,
                                      valid_actions=info['valid'],
                                      score=info['score'])
    print(f"Action: {action} -> Obs {observation}")
    return updated_state


agent = BDIAgent(plan_library=pl, nli_model=nli_model)
last_state = agent.act(current_state, step_function=step_function)

print(env.getGoalProgressStr())
print(last_state.reward)

event -> grow a orange plant from seed -> state False
event -> move to living room -> state False
Action: open door -> Obs The door is now open.
Action: go to hallway -> Obs You move to the hallway.
Action: open door to living room -> Obs The door is now open.
Action: go to living room -> Obs You move to the living room.
Action: pick up seed jar -> Obs You move the seed jar to the inventory.
event -> move to outside -> state False
Action: open door -> Obs The door is already open.
Action: go to hallway -> Obs You move to the hallway.
Action: open door to greenhouse -> Obs The door is now open.
Action: go to greenhouse -> Obs You move to the greenhouse.
Action: open door to outside -> Obs The door is now open.
Action: go to outside -> Obs You move to the outside.
event -> collect soil -> state False
Action: pick up shovel -> Obs You move the shovel to the inventory.
Action: use shovel in inventory on ground -> Obs You dig a hole in the ground, and place the soil nearby.
Action: pick up 